In [ ]:
import os
import random
import skimage.io
import skimage.transform
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from scipy import ndimage
from skimage import color
from skimage import filters

import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.optimizers import SGD
from keras.layers import Dropout
from keras.optimizers import RMSprop
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.utils import np_utils
from keras.callbacks import LearningRateScheduler, ModelCheckpoint
from keras import backend as K

# Allow image embeding in notebook
%matplotlib inline

In [ ]:
import cv2 as cv2
import tensorflow as tf

In [ ]:
def load_data(data_dir):
    """Loads a data set and returns two lists:
    images: a list of Numpy arrays, each representing an image.
    labels: a list of numbers that represent the images labels.
    This function could be used for both training data and testing data.
    """
    # Get all subdirectories of data_dir. Each represents a label.
    directories = [d for d in os.listdir(data_dir) 
                   if os.path.isdir(os.path.join(data_dir, d))]
    # Loop through the label directories and collect the data in
    # two lists, labels and images.
    labels = []
    images = []
    for d in directories:
        
        label_dir = os.path.join(data_dir, d)
        # print(label_dir) yes
        file_names = [os.path.join(label_dir, f) 
                      for f in os.listdir(label_dir) if f.endswith(".ppm")]
        # For each label, load it's images and add them to the images list.
        # And add the label number (i.e. directory name) to the labels list.
        # print(file_names)
        for f in file_names:
            print(f)
            img = skimage.io.imread(f)
            images.append(img)
            print(img.shape)
            labels.append(int(d))
    return images, labels

In [ ]:
from sklearn.model_selection import train_test_split

Load & Parse Data

In [ ]:
ROOT_PATH = "insert directory of your data"
train_data_dir = os.path.join(ROOT_PATH,"insert directory of your training data")
test_data_dir =  os.path.join(ROOT_PATH,"insert directory of your testing data")
images, labels = load_data(train_data_dir)
test_images, test_labels = load_data(test_data_dir)

Resizing Training  data to 32x32, Enter size of your choice if not 32x32

In [ ]:
images32 = [skimage.transform.resize(image, (32, 32), mode='constant')
                for image in images]

Resize Testing images,Enter size of your choice if not 32x32

In [ ]:
test_images32 = [skimage.transform.resize(img, (32, 32), mode='constant')
                for img in test_images]

Convert training and testing images to Numpy arrays

In [ ]:
image_data = np.array(images32)
print(image_data.shape)
label_data = np.array(labels)
print(label_data.shape)

In [ ]:
test_image_data = np.array(test_images32)
print(test_image_data.shape)
test_label_data = np.array(test_labels)
print(test_label_data.shape)

Train,test and split training data

In [ ]:
X_train, X_val, y_train,y_val = train_test_split(image_data, label_data,test_size=0.3, random_state=0,shuffle=True)


print(X_train.shape)
print(X_val.shape)
print(y_train.shape)
print(y_val.shape)


In [ ]:
y_train = keras.utils.to_categorical(y_train,"enter numerical value of classes in your dataset")
y_val = keras.utils.to_categorical(y_val,"enter numerical value of classes in your dataset")


print(y_train.shape)
print(y_val.shape)

Train,test and split testing data

In [ ]:
X_test, X_valid, y_test, y_valid=train_test_split(test_image_data, test_label_data, random_state=0,shuffle=True)

In [ ]:
print(X_test.shape)
print(X_valid.shape)
print(y_test.shape)
print(y_valid.shape)

In [ ]:
y_test = keras.utils.to_categorical(y_test,"enter numerical value of classes in your dataset")
y_valid = keras.utils.to_categorical(y_valid,"enter numerical value of classes in your dataset")


print(y_test.shape)
print(y_valid.shape)

Expand Training data

In [ ]:
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.data import AUTOTUNE
from imutils import paths
import matplotlib.pyplot as plt
import tensorflow as tf
import argparse
import os
from tensorflow.keras.preprocessing.image  import ImageDataGenerator

In [ ]:
data_augmentation  = ImageDataGenerator(
		                 rotation_range = 20,
										 zoom_range = 0.15,
										 width_shift_range = 0.1,
										 horizontal_flip = True,
										 vertical_flip = True,
                     featurewise_center=True,
										 fill_mode ="nearest")

NN layer

In [ ]:
model = keras.models.Sequential([
    keras.layers.Conv2D(filters =32, kernel_size=(3,3),activation='relu',input_shape =X_train.shape[1:]),
    keras.layers.Conv2D(filters =32, kernel_size=(3,3),activation='relu'),
    keras.layers.MaxPool2D(pool_size=(2,2)),
    keras.layers.Dropout(rate=0.25),

    keras.layers.Conv2D(filters =64, kernel_size=(3,3),activation='relu'),
    keras.layers.Conv2D(filters =64, kernel_size=(3,3),activation='relu'),
    keras.layers.MaxPool2D(pool_size=(2,2)),
    keras.layers.Dropout(rate=0.25),

    keras.layers.Flatten(),
    keras.layers.Dense(512, activation ='relu'),
    keras.layers.BatchNormalization(axis=-1),
    keras.layers.Dropout(rate=0.5),

    keras.layers.Dense(7, activation ='softmax')

])

In [ ]:
epochs = 100
lr =0.001

optimizer = keras.optimizers.Adam(learning_rate=lr, decay = lr/(epochs*0.5))
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

Training the network for 100 epochs, change epochs and batch size according to your choice

In [ ]:
history = model.fit(data_augmentation.flow(X_train,y_train, batch_size=20),epochs=epochs, validation_data=(X_val, y_val))

Test your model

In [ ]:
model.evaluate(x=X_test, y=y_test)

3/3 [==============================] - 0s 22ms/step - loss: 0.9489 - accuracy: 0.9103


[0.9489033818244934, 0.9102563858032227]